#### Imports

In [1]:
import carla
import json
import math
import random
import time
import numpy as np
import cv2
import open3d as o3d
import queue
from matplotlib import cm
import shutil 
import os
from enum import Enum

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


#### Connect to CARLA server

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10)
world = client.get_world()
map = world.get_map()

blueprint_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [3]:
def reload_world():
    global world, map, blueprint_library, spawn_points
    world = client.reload_world()
    map = world.get_map()
    blueprint_library = world.get_blueprint_library()
    spawn_points = world.get_map().get_spawn_points()

def load_world(map_name="Town01", timeout=10.0):
    global world, map, blueprint_library, spawn_points
    client.set_timeout(timeout)
    world = client.load_world(map_name)
    map = world.get_map()
    blueprint_library = world.get_blueprint_library()
    spawn_points = world.get_map().get_spawn_points()

In [4]:
class CarlaSyncMode(object):
    """
        with CarlaSyncMode(world, sensors) as sync_mode:
            while True:
                data = sync_mode.tick(timeout=1.0)

    """

    def __init__(self, world, sensors, **kwargs):
        self.world = world
        self.sensors = sensors
        self.frame = None
        self.delta_seconds = 1.0 / kwargs.get('fps', 5)
        self._queues = []
        self._settings = None

    def __enter__(self):
        self._settings = self.world.get_settings()
        self.frame = self.world.apply_settings(carla.WorldSettings(
            no_rendering_mode=False,
            synchronous_mode=True,
            fixed_delta_seconds=self.delta_seconds,
            ))

        def make_queue(register_event):
            q = queue.Queue()
            register_event(q.put)
            self._queues.append(q)

        make_queue(self.world.on_tick)
        for sensor in self.sensors:
            make_queue(sensor.listen)
        return self

    def tick(self, timeout):
        self.frame = self.world.tick()
        data = [self._retrieve_data(q, timeout) for q in self._queues]
        assert all(x.frame == self.frame for x in data)
        return data

    def __exit__(self, *args, **kwargs):
        self.world.apply_settings(self._settings)

    def _retrieve_data(self, sensor_queue, timeout):
        while True:
            data = sensor_queue.get(timeout=timeout)
            if data.frame == self.frame:
                return data

#### Read simulation configuration file

In [5]:
def get_value(data_dict, keys, default=None):
    for key in keys:
        if isinstance(data_dict, dict):
            data_dict = data_dict.get(key, default)
        else:
            return default
    return data_dict


configration_filepath = "config/nuscenes.config.godmode.json"
with open(configration_filepath, 'r', encoding='utf-8') as configration_file:
    configuration = json.load(configration_file)
print(json.dumps(configuration, indent=4))

{
    "robot": {
        "name": "example_robot",
        "link": [
            {
                "link": {
                    "name": "CAM_BACK_RIGHT"
                }
            },
            {
                "link": {
                    "name": "RADAR_FRONT_LEFT"
                }
            },
            {
                "link": {
                    "name": "RADAR_FRONT_RIGHT"
                }
            },
            {
                "link": {
                    "name": "base_link"
                }
            },
            {
                "link": {
                    "name": "RADAR_FRONT"
                }
            },
            {
                "link": {
                    "name": "RADAR_BACK_LEFT"
                }
            },
            {
                "link": {
                    "name": "LIDAR_TOP"
                }
            },
            {
                "link": {
                    "name": "CAM_FRONT"
                }
            },


### Apply configuration

In [6]:
def get_transform(data_dict, offsets=dict()):
    x = get_value(data_dict, ["joint", "origin", "x"], default=0) + offsets.get("x", 0)
    y = get_value(data_dict, ["joint", "origin", "y"], default=0) + offsets.get("y", 0)
    z = get_value(data_dict, ["joint", "origin", "z"], default=0) + offsets.get("z", 0)
    roll = get_value(data_dict, ["joint", "origin", "roll"], default=0)
    roll = math.degrees(roll) + offsets.get("roll", 0)
    pitch = get_value(data_dict, ["joint", "origin", "pitch"], default=0)
    pitch = math.degrees(pitch) + offsets.get("pitch", 0)
    yaw = get_value(data_dict, ["joint", "origin", "yaw"], 0)
    yaw = math.degrees(yaw) + offsets.get("yaw", 0)
    return carla.Transform(carla.Location(x=x, y=y, z=z), carla.Rotation(roll=roll, pitch=pitch, yaw=yaw))

def rotate_around_z_axis(transform, degrees):
    x = transform.location.x#*math.cos(degrees) - transform.location.y*math.sin(degrees)
    y = transform.location.y#transform.location.x*math.sin(degrees) + transform.location.y*math.cos(degrees)
    z = transform.location.z
    roll = transform.rotation.roll
    pitch = transform.rotation.pitch
    yaw = transform.rotation.yaw + degrees
    return carla.Transform(carla.Location(x=x, y=y, z=z), carla.Rotation(roll=roll, pitch=pitch, yaw=yaw))

reload_world()

vehicles = []
blueprint_name = "vehicle.kawasaki.ninja"
blueprint = blueprint_library.find(blueprint_name)
blueprint.set_attribute('role_name','ego')
transform = spawn_points[20]
vehicle = world.spawn_actor(blueprint, transform)
vehicles.append(vehicle)

sensor_names = []
sensor_types = []
sensors = []
sensor_configuration_list = get_value(configuration, ["robot", "joint"], default=[])
for sensor_configuration in sensor_configuration_list:
    sensor_name = get_value(sensor_configuration, ["joint", "name"])

    if "_DEPTH_CAM_" in sensor_name:
        blueprint_name = "sensor.camera.depth"
        blueprint = blueprint_library.find(blueprint_name)
        sensor_intrinsics = get_value(sensor_configuration, ["joint", "intrinsics"], default=dict())
        image_width = str(sensor_intrinsics.get("w", 1600))
        blueprint.set_attribute('image_size_x', image_width)
        image_height = str(sensor_intrinsics.get("h", 900))
        field_of_view = str(sensor_intrinsics.get("fov", 120))
        blueprint.set_attribute('image_size_y', image_height)
        blueprint.set_attribute('fov', field_of_view)
        transform = get_transform(sensor_configuration, dict(z=0.4, roll=90))
        transform = rotate_around_z_axis(transform, 90)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)

        sensors.append(sensor)
        sensor_type = blueprint_name
        sensor_types.append(sensor_type)
        sensor_name = get_value(
            sensor_configuration, ["joint", "name"], 
            default=f"Vehicle | {blueprint_name} | Location=({transform.location}), Rotation=({transform.rotation})")
        sensor_names.append(sensor_name)

    elif "_SEMANTIC_CAM_" in sensor_name:
        blueprint_name = "sensor.camera.semantic_segmentation"
        blueprint = blueprint_library.find(blueprint_name)
        sensor_intrinsics = get_value(sensor_configuration, ["joint", "intrinsics"], default=dict())
        image_width = str(sensor_intrinsics.get("w", 1600))
        blueprint.set_attribute('image_size_x', image_width)
        image_height = str(sensor_intrinsics.get("h", 900))
        field_of_view = str(sensor_intrinsics.get("fov", 120))
        blueprint.set_attribute('image_size_y', image_height)
        blueprint.set_attribute('fov', field_of_view)
        transform = get_transform(sensor_configuration, dict(z=0.4, roll=90))
        transform = rotate_around_z_axis(transform, 90)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)

        sensors.append(sensor)
        sensor_type = blueprint_name
        sensor_types.append(sensor_type)
        sensor_name = get_value(
            sensor_configuration, ["joint", "name"], 
            default=f"Vehicle | {blueprint_name} | Location=({transform.location}), Rotation=({transform.rotation})")
        sensor_names.append(sensor_name)

    elif "_CAM_" in sensor_name:
        blueprint_name = "sensor.camera.rgb"
        blueprint = blueprint_library.find(blueprint_name)
        sensor_intrinsics = get_value(sensor_configuration, ["joint", "intrinsics"], default=dict())
        focal_distance = str(sensor_intrinsics.get("fl", 1260.0))
        blueprint.set_attribute("focal_distance", focal_distance)
        image_width = str(sensor_intrinsics.get("w", 1600))
        blueprint.set_attribute('image_size_x', image_width)
        image_height = str(sensor_intrinsics.get("h", 900))
        field_of_view = str(sensor_intrinsics.get("fov", 120))
        blueprint.set_attribute('image_size_y', image_height)
        blueprint.set_attribute('fov', field_of_view)
        transform = get_transform(sensor_configuration, dict(z=0.4, roll=90))
        transform = rotate_around_z_axis(transform, 90)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)

        sensors.append(sensor)
        sensor_type = blueprint_name
        sensor_types.append(sensor_type)
        sensor_name = get_value(
            sensor_configuration, ["joint", "name"], 
            default=f"Vehicle | {blueprint_name} | Location=({transform.location}), Rotation=({transform.rotation})")
        sensor_names.append(sensor_name)

    elif "_RADAR_" in sensor_name:
        blueprint_name = "sensor.other.radar"
        blueprint = blueprint_library.find(blueprint_name)     
        blueprint.set_attribute('horizontal_fov', str(360.0))   
        transform = get_transform(sensor_configuration, dict(z=0.4))
        transform = rotate_around_z_axis(transform, 90)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)

        sensors.append(sensor)
        sensor_type = blueprint_name
        sensor_types.append(sensor_type)
        sensor_name = get_value(
            sensor_configuration, ["joint", "name"], 
            default=f"Vehicle | {blueprint_name} | Location=({transform.location}), Rotation=({transform.rotation})")
        sensor_names.append(sensor_name)

    elif "_LIDAR_" in sensor_name:
        blueprint_name = "sensor.lidar.ray_cast_semantic"
        blueprint = blueprint_library.find(blueprint_name)
        blueprint.set_attribute("channels", str(64))
        blueprint.set_attribute("points_per_second", str(112000))
        blueprint.set_attribute("range", str(100))
        transform = get_transform(sensor_configuration, dict(z=0.4))
        transform = rotate_around_z_axis(transform, 90)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)
        
        sensors.append(sensor)
        sensor_type = blueprint_name
        sensor_types.append(sensor_type)
        sensor_name = get_value(
            sensor_configuration, ["joint", "name"], 
            default=f"Vehicle | {blueprint_name} | Location=({transform.location}), Rotation=({transform.rotation})")
        sensor_names.append(sensor_name)



In [7]:
print(sensor_names)
print(sensor_types)

['base_link_to_RADAR_FRONT', 'base_link_to_RADAR_FRONT_LEFT', 'base_link_to_RADAR_FRONT_RIGHT', 'base_link_to_RADAR_BACK_LEFT', 'base_link_to_RADAR_BACK_RIGHT', 'base_link_to_LIDAR_TOP', 'base_link_to_CAM_FRONT', 'base_link_to_CAM_FRONT_RIGHT', 'base_link_to_CAM_BACK_RIGHT', 'base_link_to_CAM_BACK', 'base_link_to_CAM_BACK_LEFT', 'base_link_to_CAM_FRONT_LEFT', 'base_link_to_DEPTH_CAM_FRONT', 'base_link_to_DEPTH_CAM_FRONT_RIGHT', 'base_link_to_DEPTH_CAM_BACK_RIGHT', 'base_link_to_DEPTH_CAM_BACK', 'base_link_to_DEPTH_CAM_BACK_LEFT', 'base_link_to_DEPTH_CAM_FRONT_LEFT', 'base_link_to_SEMANTIC_CAM_FRONT', 'base_link_to_SEMANTIC_CAM_FRONT_RIGHT', 'base_link_to_SEMANTIC_CAM_BACK_RIGHT', 'base_link_to_SEMANTIC_CAM_BACK', 'base_link_to_SEMANTIC_CAM_BACK_LEFT', 'base_link_to_SEMANTIC_CAM_FRONT_LEFT']
['sensor.other.radar', 'sensor.other.radar', 'sensor.other.radar', 'sensor.other.radar', 'sensor.other.radar', 'sensor.lidar.ray_cast_semantic', 'sensor.camera.rgb', 'sensor.camera.rgb', 'sensor.cam

### Helper methods:

##### Filesystem methods

In [8]:
def delete_all_in_directory(target_directory):
    if os.path.exists(target_directory):
        for filename in os.listdir(target_directory):
            file_path = os.path.join(target_directory, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.remove(file_path) 
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')


def create_filename_from_timestamp(timestamp):
    SECONDS_TO_NANOSECONDS = 1000000000
    filename = str(math.trunc(timestamp * SECONDS_TO_NANOSECONDS))
    return filename


##### General sensor processing methods

In [9]:
def save_sensor_position(raw_data, target_directory):
    transform = raw_data.transform
    transform_matrix = raw_data.transform.get_matrix()
    if not os.path.exists(target_directory):
        os.makedirs(target_directory, exist_ok=True)
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".npy"
    np.save(f"{target_directory}/{filename}", transform_matrix)

##### RBG Camera Methods

In [10]:
def save_rgb_image(raw_data, target_directory):    
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    rgb_image = np.reshape(np.copy(raw_data.raw_data), (raw_data.height, raw_data.width, 4))
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".png"
    cv2.imwrite(f"{directory}/{filename}", rgb_image)

def visualize_rgb_image(raw_image, window_name, window_size=None):
    image = np.reshape(np.copy(raw_image.raw_data), (raw_image.height, raw_image.width, 4))
    if window_size:
         image = cv2.resize(image, dsize=window_size, interpolation=cv2.INTER_CUBIC)
    cv2.imshow(window_name, image)

##### Depth Camera Methods

In [11]:
def visualize_depth_image(raw_image, window_name, window_size=None):
    image = np.reshape(np.copy(raw_image.raw_data), (raw_image.height, raw_image.width, 4))
    if window_size:
         image = cv2.resize(image, dsize=window_size, interpolation=cv2.INTER_CUBIC)
    cv2.imshow(window_name, image)

def save_depth_image(raw_data, target_directory):
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
        
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".png"
    raw_data.save_to_disk(f"{directory}/{filename}")

def convert_depth_image_to_depth_map(image):
    depth_map = image[:, :, 0] + image[:, :, 1] * 256 + image[:, :, 2] * 256**2
    depth_map = depth_map / (256**3 - 1) * 1000  # Convert to meters
    return depth_map

##### Semantic Camera Methods

In [12]:
class SemanticColors(Enum):
    UNLABELED =     (0, 0, 0)
    BUILDING =      (1, 0, 0)
    FENCE =         (2, 0, 0)
    OTHER =         (3, 0, 0)
    PEDESTRIAN =    (4, 0, 0)
    POLE =          (5, 0, 0)
    ROADLINE =      (6, 0, 0)
    ROAD =          (7, 0, 0)
    SIDEWALK =      (8, 0, 0)
    VEGETATION =    (9, 0, 0)
    VEHICLES =      (10, 0, 0)
    WALL =          (11, 0, 0)
    TRAFFICSIGN =   (12, 0, 0)
    SKY =           (13, 0, 0)
    GROUND =        (14, 0, 0)
    BRIDGE =        (15, 0, 0)
    RAILTRACK =     (16, 0, 0)
    GUARDRAIL =     (17, 0, 0)
    TRAFFICLIGHT =  (18, 0, 0)
    STATIC =        (19, 0, 0)
    DYNAMIC =       (20, 0, 0)
    WATER =         (21, 0, 0)
    TERRAIN =       (22, 0, 0)

def create_color_mask(image, colors):
    mask = np.full((image.shape[0], image.shape[1]), 255, dtype=np.uint8)
    
    B, G, R = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    # Iterate through the list of colors
    for color in colors:
        # Extract color channels
        r, g, b = color
        # Create boolean masks for each channel comparison
        r_mask = R == r
        g_mask = G == g
        b_mask = B == b
        # Combine channel masks to get the final color mask
        color_mask = r_mask & g_mask & b_mask
        # Update the overall mask where any color matches
        mask[color_mask] = 0
    return mask

def visualize_semantic_image(raw_image, window_name, window_size=None):
    image = np.reshape(np.copy(raw_image.raw_data), (raw_image.height, raw_image.width, 4))
    if window_size:
         image = cv2.resize(image, dsize=window_size, interpolation=cv2.INTER_CUBIC)
    cv2.imshow(window_name, image)

def save_semantic_image(raw_data, target_directory):
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".png"
    raw_data.save_to_disk(f"{directory}/{filename}")

def save_semantic_image_mask(raw_data, target_directory, masked_colors=[]):
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

    semantic_image = np.reshape(np.copy(raw_data.raw_data), (raw_data.height, raw_data.width, 4))
    mask = create_color_mask(semantic_image, masked_colors)

    filename = create_filename_from_timestamp(raw_data.timestamp) + ".mask.png"
    cv2.imwrite(f"{directory}/{filename}", mask)

##### Radar Methods

In [13]:
def save_radar_readings(raw_data, target_directory):
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

    points = []

    # Convert radar readings to 3D points (spherical to Cartesian)
    radar_measurement = raw_data
    for detection in radar_measurement:
        # Extract the azimuth, altitude, depth from the radar detection
        azimuth = detection.azimuth
        altitude = detection.altitude
        depth = detection.depth

        # Convert spherical coordinates to Cartesian coordinates
        x = depth * np.cos(altitude) * np.sin(azimuth)
        y = depth * np.sin(altitude)
        z = depth * np.cos(altitude) * np.cos(azimuth)

        points.append([x, y, z])

    # Create an Open3D point cloud object
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    # Save the point cloud to a .ply file
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".ply"
    o3d.io.write_point_cloud(f"{directory}/{filename}", point_cloud)


def visualize_radar(raw_data, window_name, max_distance = 100, image_size=(200, 200)):
    # Get the radar data
    radar_data = raw_data.raw_data
    points = np.frombuffer(radar_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (len(raw_data), 4))

    # Extract information from radar points
    azimuths = points[:, 1]
    depths = points[:, 3]
    X = np.cos(azimuths) * depths
    Y = np.sin(azimuths) * depths

    def normalize(data):
        shifted = data - np.min(data)
        normal = shifted / np.max(shifted)
        return normal
    width, height = 360, 360
    X_norm = (normalize(X) * (width-1)).astype(int)
    Y_norm = (normalize(Y) * (height-1)).astype(int)

    radar_image_array = np.zeros((height, width))
    radar_image_array[Y_norm, X_norm] = 255  # Set a constant value for visibility
    cv2.imshow(window_name, radar_image_array)

##### Lidar methods

In [14]:
def visualize_lidar(raw_data, window_name):
    # Get the LiDAR point cloud from the data
    lidar_data = raw_data.raw_data
    lidar_data = np.frombuffer(lidar_data, dtype=np.dtype('f4'))
    lidar_data = np.reshape(lidar_data, (int(lidar_data.shape[0] / 4), 4))

    # Extract X, Y, Z coordinates and intensity values
    points_xyz = lidar_data[:, :3]
    #intensity = lidar_data[:, 3]

    # Intensity scaling factor
    intensity_scale = 2.0  # Adjust this value to control the brightness

    # Create a 2D histogram with a predetermined size
    width, height = 360, 360
    lidar_image_array = np.zeros((height, width))

    # Scale and shift X and Y coordinates to fit within the histogram size
    def normalize(data):
        shifted = data - np.min(data)
        normal = shifted / np.max(shifted)
        return normal
    X = (normalize(points_xyz[:, 0]) * (width-1)).astype(int)
    Y = (normalize(points_xyz[:, 1]) * (height-1)).astype(int)

    # Assign scaled intensity values to the corresponding pixel in the histogram
    lidar_image_array[Y, X] = 255

    # Display the processed image using Pygame
    cv2.imshow(window_name, lidar_image_array)

def save_lidar_readings(raw_data, target_directory):
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".ply"
    raw_data.save_to_disk(f"{directory}/{filename}")

##### Semantic Lidar Methods

In [15]:
def filter_semantic_lidar_readings(semantic_lidar_data, filters=dict()):
    #print(np.unique(semantic_tags, return_counts=True))
    filtered_lidar_data = np.copy(semantic_lidar_data)
    
    tags = filters.get("tags", None)
    if tags:
        data_tags = filtered_lidar_data[:, 3:4].flatten()
        filter_mask = np.isin(data_tags, tags)
        filtered_lidar_data = filtered_lidar_data[filter_mask]
        #print(len(semantic_lidar_data), len(filtered_lidar_data))
    
    z_range = filters.get("z_range", None)
    if z_range:
        z_min, z_max = z_range
        z_values = filtered_lidar_data[:, 2:3].flatten()
        filter_mask = np.where(z_min <= z_values, True, False) * np.where(z_values <= z_max, True, False)
        filtered_lidar_data = filtered_lidar_data[filter_mask]
    
    return filtered_lidar_data

def visualize_semantic_lidar(raw_data, window_name):
    # Get the LiDAR point cloud from the data
    lidar_data = np.array([(detection.point.x, detection.point.y, detection.point.z, int(detection.object_tag)) for detection in raw_data])
    lidar_data = filter_semantic_lidar_readings(lidar_data, filters=dict(
        z_range=(-1, 10), 
        tags=[1,2,3,4,5,9,10,11,12,13,15,16,17,18,19,20,21]
    ))

    # Extract X, Y, Z coordinates and intensity values
    points_xyz = lidar_data[:, :3]
    #intensity = lidar_data[:, ::3]

    # Intensity scaling factor
    intensity_scale = 0.2  # Adjust this value to control the brightness

    # Create a 2D histogram with a predetermined size
    width, height = 360, 360
    lidar_image_array = np.zeros((height, width))

    # Scale and shift X and Y coordinates to fit within the histogram size
    def normalize(data):
        shifted = data - np.min(data)
        normal = shifted / np.max(shifted)
        return normal
    X = (normalize(points_xyz[:, 0]) * (width-1)).astype(int)
    Y = (normalize(points_xyz[:, 1]) * (height-1)).astype(int)

    # Assign scaled intensity values to the corresponding pixel in the histogram
    lidar_image_array[Y, X] = 255

    # Display the processed image using Pygame
    cv2.imshow(window_name, lidar_image_array)

def save_semantic_lidar_readings(raw_data, target_directory):
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

    lidar_data = np.array([(detection.point.x, detection.point.y, detection.point.z, int(detection.object_tag)) for detection in raw_data])
    lidar_data = filter_semantic_lidar_readings(lidar_data, filters=dict(
        z_range=(-2, 10),
        tags=[
            1,2,5,9,10,11,12,14   #1,2,3,4,5,9,10,11,12,13,15,16,17,18,19,20,21
        ]
    ))
    points = lidar_data[:, :3]
    # Create an Open3D point cloud object
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    # Save the point cloud to a .ply file
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".ply"
    o3d.io.write_point_cloud(f"{target_directory}/{filename}", point_cloud)

#### Running the simulation

In [16]:
SIMULATED_DATA_DIRECTORY = "./generated_data"
delete_all_in_directory(SIMULATED_DATA_DIRECTORY)

cv2.namedWindow("Press Q to stop simulation")
cv2.imshow("Press Q to stop simulation", np.zeros((1,1)))

try:
    with CarlaSyncMode(world, sensors, fps=10) as sync_mode:
        while True:
            simulation_results = sync_mode.tick(timeout=300.0)[1:]
            for i in range(len(simulation_results)):
                sensor = sensors[i]
                sensor_data = simulation_results[i]
                sensor_name = sensor_names[i].replace("base_link_to_", "")
                sensor_type = sensor_types[i]
                if ("camera.rgb" in sensor_type):
                    #visualize_rgb_image(sensor_data, sensor_name, window_size = (480, 270))
                    save_rgb_image(sensor_data, f"generated_data/{sensor_name}/")
                elif ("camera.semantic_segmentation" in sensor_type):
                    #visualize_semantic_image(sensor_data, sensor_name, window_size = (480, 270))
                    save_semantic_image(sensor_data, f"generated_data/{sensor_name}/")
                    save_semantic_image_mask(sensor_data, f"generated_data/{sensor_name}/", masked_colors=[
                        SemanticColors.ROADLINE.value, SemanticColors.ROAD.value, SemanticColors.SIDEWALK.value,
                        SemanticColors.GROUND.value, SemanticColors.WATER.value, SemanticColors.TERRAIN.value,
                        SemanticColors.SKY.value
                    ])
                elif ("sensor.other.radar" in sensor_type):
                    #visualize_radar(sensor_data, sensor_name)
                    save_radar_readings(sensor_data, f"generated_data/{sensor_name}/")
                elif ("camera.depth" in sensor_type):
                    #visualize_depth_image(sensor_data, sensor_name, window_size = (480, 270))
                    save_depth_image(sensor_data, f"generated_data/{sensor_name}/") 
                elif ("sensor.lidar.ray_cast_semantic" in sensor_type):
                    #visualize_semantic_lidar(sensor_data, sensor_name)
                    save_semantic_lidar_readings(sensor_data, f"generated_data/{sensor_name}/")
                elif ("sensor.lidar" in sensor_type):
                    #visualize_lidar(sensor_data, sensor_name)
                    save_lidar_readings(sensor_data, f"generated_data/{sensor_name}/")
                save_sensor_position(sensor_data, f"generated_data/{sensor_name}/")
            for vehicle in vehicles:
                position = map.get_waypoint(vehicle.get_transform().location)
                next_postition = random.choice(position.next(1.5))
                vehicle.set_transform(next_postition.transform)
            
            if cv2.waitKey(1) == ord('q'):
                break
finally:
    cv2.destroyAllWindows()
    for sensor in sensors:
        sensor.stop()